## Imports

In [1]:
import MDAnalysis as mda
from MDAnalysis.analysis.rdf import InterRDF
from MDAnalysis.transformations import NoJump
from MDAnalysis.analysis import msd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress
import csv, re 
from scipy.ndimage import gaussian_filter1d
from scipy.signal.windows import hann
from scipy.interpolate import UnivariateSpline

## Analysis Functions 
## 1 - RDF

In [ ]:
# --- Function: do RDF from lammps XYZ output as obtained from imput scripts in the MLP simulations section#
def do_rdf(file, atom_1='name O', atom_2='name O', nbins=300, rdf_range=(0.0, 10.0), box=12.429):
    """do RDF from lammps XYZ output as obtained from imput scripts in the MLP simulations section"""
    try:
        u = mda.Universe(file, format='XYZ')

        # Select only oxygen atoms (assuming '1' denotes oxygens)
        for atom in u.atoms:
            if atom.name == '2':
                atom.name = 'O'
                atom.type = 'O'
                atom.mass = 15.999
            elif atom.name == '1':
                atom.name = 'H'
                atom.type = 'H'
                atom.mass = 1.008

        bonds = []
        for i in range(0, len(u.atoms), 3):
            oxygen = u.atoms[i]
            hydrogen1 = u.atoms[i + 1]
            hydrogen2 = u.atoms[i + 2]
            bonds.extend([(oxygen.index, hydrogen1.index), (oxygen.index, hydrogen2.index)])

        u.add_bonds(bonds)
        if not np.any(u.dimensions):
            u.dimensions = [box, box, box, 90, 90, 90]

        sel_1 = u.select_atoms(atom_1)
        sel_2 = u.select_atoms(atom_2)
        allat = u.select_atoms('all')

        # Trajectory unwrapping (ensure proper handling of PBC)
        transf = NoJump(allat)
        u.trajectory.add_transformations(transf)

        # Setup the RDF calculation
        rdf_calculator = InterRDF(sel_1, sel_2, range=rdf_range, exclusion_block=(1, 1), nbins=nbins)

        # Run the RDF calculation across the trajectory
        try:
            rdf_calculator.run()
        except IndexError as e:
            print(f"IndexError in RDF calculation for file {file}: {e}")
            return None  # Skip problematic file

        return rdf_calculator
    except Exception as e:
        print(f"Error processing file {file}: {e}")
        return None
#Example Usage 
#DFT_RDF        = do_rdf('trajectory.xyz',box=24.75)

## Analysis Functions 
## 2 - Diffiusion Coefficient

In [ ]:
def compute_diffusion_coefficient(
    xyz_files,
    box_length_A,
    start_ps,
    end_ps,
    timestep_ps=0.1,
    selection="name O",
    dim_factor=3,
    temperature=300.0,
    viscosity=0.89e-3,
    xi=2.837297,
    n_bootstrap=1000,
    plot_msd=True
):
    """
    Compute the diffusion coefficient from a list of .xyz files using MDAnalysis,
    including a finite-size (Yeh–Hummer) correction and bootstrap error estimation.

    Parameters
    ----------
    xyz_files : list of str
        Paths to the .xyz trajectory files.
    box_length_A : float
        Cubic box length in Angstroms (Å).
    start_ps : float
        Start time (ps) of the linear fit window (in the diffusive regime).
    end_ps : float
        End time (ps) of the linear fit window.
    timestep_ps : float, optional
        Time step between frames in ps (default 0.1 ps).
    selection : str, optional
        Atom selection for MSD calculation (default "name O").
    dim_factor : int, optional
        Dimensional factor for the Einstein relation (3 for 3D, 2 for 2D, etc.).
    temperature : float, optional
        Temperature in K for the finite-size correction (default 300 K).
    viscosity : float, optional
        Fluid viscosity in Pa·s (default 0.89e-3, typical for water at ~300 K).
    xi : float, optional
        Geometry constant for cubic boxes in the Yeh–Hummer correction (default 2.837297).
    n_bootstrap : int, optional
        Number of bootstrap resamples (default 1000).
    plot_msd : bool, optional
        If True, plot both the individual trajectory MSDs and the overall averaged MSD (default True).

    Returns
    -------
    results : dict
        A dictionary containing:
        - "D_raw_A2_ps": Diffusion coefficient from the linear fit in Å^2/ps
        - "D_raw_err_A2_ps": Uncertainty in D_raw in Å^2/ps (from bootstrap)
        - "D_infinity_m2_s": Finite-size corrected D in m^2/s
        - "D_infinity_err_m2_s": Uncertainty in corrected D in m^2/s
        - "avg_msd": Array of the averaged MSD (Å^2)
        - "lagtimes": Array of time points (ps) for the MSD
    """
    # ------------------
    # 1) Read trajectories, compute MSD
    # ------------------
    all_msds = []
    frame_counts = []

    for xyz in xyz_files:
        # Create universe from XYZ
        u = mda.Universe(xyz, format='XYZ')
        
        # Rename atoms
        for atom in u.atoms:
            if atom.name == '2':
                atom.name = 'O'
                atom.type = 'O'
                atom.mass = 15.999
            elif atom.name == '1':
                atom.name = 'H'
                atom.type = 'H'
                atom.mass = 1.008
        
        # Add bonds for each water (3 atoms)
        bonds = []
        for i in range(0, len(u.atoms), 3):
            oxygen = u.atoms[i]
            hydrogen1 = u.atoms[i+1]
            hydrogen2 = u.atoms[i+2]
            bonds.extend([
                (oxygen.index, hydrogen1.index),
                (oxygen.index, hydrogen2.index)
            ])
        u.add_bonds(bonds)
        
        # Set dimensions if missing
        if not np.any(u.dimensions):
            u.dimensions = [box_length_A, box_length_A, box_length_A, 90, 90, 90]
        
        # Apply NoJump unwrapping
        u.trajectory.add_transformations(NoJump())
        
        # Compute MSD for selection
        msd_analysis = msd.EinsteinMSD(
            u,
            select=selection,
            msd_type="xyz",
            fft=True
        )
        msd_analysis.run()
        
        # (n_frames, n_particles)
        msds_array = msd_analysis.results.msds_by_particle
        all_msds.append(msds_array)
        frame_counts.append(msds_array.shape[0])

    # Truncate all MSD arrays to the length of the shortest trajectory
    min_frames = min(frame_counts)
    all_msds_trunc = [arr[:min_frames, :] for arr in all_msds]

    # Combine replicate MSDs (concatenate along axis=1)
    combined_msds = np.concatenate(all_msds_trunc, axis=1)

    # Average over all particles (across all trajectories)
    avg_msd = np.mean(combined_msds, axis=1)  # shape: (min_frames,)
    lagtimes = np.arange(min_frames) * timestep_ps

    # ------------------
    # Plotting: individual trajectories and overall average
    # ------------------
    if plot_msd:
        plt.figure()
        # Plot each trajectory's average MSD (averaging over particles in that trajectory)
        for i, msd_arr in enumerate(all_msds_trunc):
            traj_avg = np.mean(msd_arr, axis=1)
            plt.plot(lagtimes, traj_avg, linestyle='--', alpha=0.7, label=f"Trajectory {i+1}")
        # Overlay the overall average MSD
        plt.plot(lagtimes, avg_msd, color='black', linewidth=2, label="Overall Average")
        plt.xlabel("Lag time (ps)")
        plt.ylabel("MSD (Å$^2$)")
        plt.legend()
        plt.tight_layout()
        plt.show()

    # ------------------
    # 2) Linear Fit in the Diffusive Regime
    # ------------------
    start_idx = np.searchsorted(lagtimes, start_ps)
    end_idx = np.searchsorted(lagtimes, end_ps)
    
    # Ordinary least squares on the average
    lr = linregress(lagtimes[start_idx:end_idx], avg_msd[start_idx:end_idx])
    slope = lr.slope
    # D in Å^2/ps
    D_raw_A2_ps = slope / (2.0 * dim_factor)

    # ------------------
    # 3) Bootstrap to get error
    # ------------------
    n_particles = combined_msds.shape[1]
    bootstrap_slopes = np.zeros(n_bootstrap)

    for i in range(n_bootstrap):
        # Resample with replacement among particles
        sample_indices = np.random.choice(n_particles, n_particles, replace=True)
        boot_avg_msd = np.mean(combined_msds[:, sample_indices], axis=1)
        
        lr_boot = linregress(lagtimes[start_idx:end_idx], boot_avg_msd[start_idx:end_idx])
        bootstrap_slopes[i] = lr_boot.slope

    # Mean slope and std from bootstrap
    slope_mean = np.mean(bootstrap_slopes)
    slope_std = np.std(bootstrap_slopes)

    D_bootstrap_A2_ps = slope_mean / (2.0 * dim_factor)
    D_bootstrap_err_A2_ps = slope_std / (2.0 * dim_factor)

    # ------------------
    # 4) Finite-Size Correction (Yeh–Hummer)
    # ------------------
    # Convert from Å^2/ps to m^2/s
    # 1 Å = 1e-10 m, 1 ps = 1e-12 s => 1 (Å^2/ps) = 1e-8 (m^2/s)
    D_raw_m2_s = D_bootstrap_A2_ps * 1.0e-8
    D_raw_err_m2_s = D_bootstrap_err_A2_ps * 1.0e-8

    # Box length in meters
    L_m = box_length_A * 1.0e-10

    # k_B in J/K
    kB = 1.380649e-23
    beta = 1.0 / (kB * temperature)

    # Yeh–Hummer correction term: D(∞) = D(L) + xi/(6π η β L)
    D_correction_m2_s = xi / (6.0 * np.pi * viscosity * beta * L_m)

    # Final corrected D
    D_infinity_m2_s = D_raw_m2_s + D_correction_m2_s

    # Assuming the correction is exact (no error in T, eta, L, xi),
    # the error in D(∞) is the same as that in D(L).
    D_infinity_err_m2_s = D_raw_err_m2_s

    # ------------------
    # 5) Prepare Results
    # ------------------
    results = {
        "D_raw_A2_ps": D_bootstrap_A2_ps,
        "D_raw_err_A2_ps": D_bootstrap_err_A2_ps,
        "D_infinity_m2_s": D_infinity_m2_s,
        "D_infinity_err_m2_s": D_infinity_err_m2_s,
        "avg_msd": avg_msd,
        "lagtimes": lagtimes
    }

    # Print summary
    print("=== Diffusion Coefficient Results ===")
    print(f"Raw D(L) in Å^2/ps = {D_bootstrap_A2_ps:.5f} ± {D_bootstrap_err_A2_ps:.5f}")
    print(f"Raw D(L) in m^2/s  = {D_raw_m2_s:.4e} ± {D_raw_err_m2_s:.4e}")
    print(f"Correction term    = {D_correction_m2_s:.4e} m^2/s")
    print(f"D(∞) in m^2/s      = {D_infinity_m2_s:.4e} ± {D_infinity_err_m2_s:.4e}")

    return results

# Example usage
#xyz_files = [f'run_nve_{i}.xyz' for i in range(1,11)]
#
# Suppose your box is 25 Å on each side
# box_length_A = 25

# We'll fit from 10 ps to 50 ps
#start_ps = 10.0
#end_ps   = 50.0
#
#results = compute_diffusion_coefficient(
#    xyz_files,
#    box_length_A,
#    start_ps,
#    end_ps,
#    timestep_ps=0.5,
#    selection="name O",
#    dim_factor=3,
#    temperature=300.0,
#    viscosity=0.89e-3,
#    xi=2.837297,
#    n_bootstrap=1000,
#    plot_msd=True
#)

## Analysis Functions 
## 3 - Fusion and Vaporization Enthalpies

In [31]:
def read_lammps_log(filename, time_step=None):
    data_lines = []
    header = None
    with open(filename, 'r') as file:
        lines = file.readlines()
        i = 0
        while i < len(lines):
            line = lines[i]
            if line.strip().startswith('Step'):
                header = line.strip().split()
                i += 1
                while i < len(lines):
                    line = lines[i].strip()
                    if not line:
                        break
                    tokens = line.split()
                    if len(tokens) != len(header):
                        break
                    try:
                        data_line = [float(token) for token in tokens]
                        data_lines.append(data_line)
                    except ValueError:
                        break
                    i += 1
            else:
                i += 1
    if not data_lines:
        raise ValueError('No data found in the log file.')
    data = np.array(data_lines)
    if 'Time' not in header:
        if time_step is None:
            time_step = 0.0005  # ps per step; adjust if necessary
        if 'Step' not in header:
            raise ValueError("'Step' column not found in data.")
        step_idx = header.index('Step')
        time_array = data[:, step_idx] * time_step
        header.append('Time')
        data = np.hstack((data, time_array.reshape(-1, 1)))
    return header, data

def read_ase_log(filename):
    data_lines = []
    header = ['Step', 'Time', 'Temp', 'Press', 'Density', 'E_pair', 'TotEng']
    time_step = 0.0005  # ps per step; adjust if necessary
    step_offset = 0
    prev_step = None
    with open(filename, 'r') as file:
        lines = file.readlines()
        for line in lines:
            line = line.strip()
            if line.startswith('Step:'):
                line = re.sub(r'([a-zA-Z]):', r'\1: ', line)
                line = re.sub(r'([a-zA-Z])([0-9])', r'\1 \2', line)
                parts = line.split(',')
                data_dict = {}
                for part in parts:
                    if ':' in part:
                        key, value = part.split(':', 1)
                        key = key.strip()
                        value = value.strip().split()[0]
                        try:
                            data_dict[key] = float(value)
                        except ValueError:
                            continue
                current_step = data_dict.get('Step', 0)
                if prev_step is not None and current_step < prev_step:
                    step_offset += prev_step + 1  # adjust for restart
                adjusted_step = current_step + step_offset
                prev_step = current_step
                time_val = adjusted_step * time_step
                density = data_dict.get('Density', np.nan)
                data_line = [
                    adjusted_step,
                    time_val,
                    data_dict.get('Temperature', np.nan),
                    data_dict.get('Pressure', np.nan),
                    density,
                    data_dict.get('Mace Energy', np.nan),
                    data_dict.get('Total Energy', np.nan)
                ]
                data_lines.append(data_line)
    if not data_lines:
        raise ValueError('No data found in the ASE log file.')
    data = np.array(data_lines)
    return header, data

def compute_average_enthalpy(log_type, filename, energy_column, time_step=0.0005,
                             conversion_factor=6.24e-7, ase_mass=None, eq_window=None,
                             num_molecules=None):
    """
    Compute the average enthalpy from a log file and convert the result to meV/mol.

    Parameters:
      log_type (str): Either 'lammps' or 'ase'.
      filename (str): Path to the log file.
      energy_column (str): Name of the energy column (e.g. 'TotEng').
      time_step (float): Time per step (ps) if not present.
      conversion_factor (float): Converts (bar * Å³) to eV.
      ase_mass (float or None): For ASE logs without a Volume column, use mass to compute V = mass / Density.
      eq_window (float or None): Equilibrium window (in ps) over which to average. If None, average over all data.
      num_molecules (int or None): Number of molecules in the simulation box (required for conversion to meV/mol).

    Returns:
      avg_enthalpy_meVmol (float): The average enthalpy in meV/mol.
    """
    # Parse the file
    if log_type == 'lammps':
        header, data = read_lammps_log(filename, time_step)
    elif log_type == 'ase':
        header, data = read_ase_log(filename)
        print(header)
    else:
        raise ValueError("Unsupported log type.")

    # Map header names to indices.
    col_idx = {name: idx for idx, name in enumerate(header)}
    
    # Check for required columns.
    if 'Press' not in col_idx:
        raise ValueError("Column 'Press' not found.")
    if energy_column not in col_idx:
        raise ValueError(f"Energy column '{energy_column}' not found.")
    
    # Extract pressure (in bar) and energy (in eV).
    press_data = data[:, col_idx['Press']]
    energy_data = data[:, col_idx[energy_column]]
    
    # Determine volume: use 'Volume' if available; otherwise, compute from Density.
    if 'Volume' in col_idx:
        volume_data = data[:, col_idx['Volume']]
    elif 'Density' in col_idx:
        if ase_mass is None:
            raise ValueError("ASE log provided without 'Volume'. Please supply ase_mass to compute volume from Density.")
        density_data = data[:, col_idx['Density']]
        volume_data = ase_mass / density_data
    else:
        raise ValueError("Neither 'Volume' nor 'Density' found in the data.")
    
    # Compute the instantaneous enthalpy for each snapshot: H = E + (P*V)*conversion_factor (in eV/box).
    enthalpy = energy_data + press_data * volume_data * conversion_factor
    
    # Get the time array.
    if 'Time' in col_idx:
        time_data = data[:, col_idx['Time']]
    elif 'Step' in col_idx:
        time_data = data[:, col_idx['Step']] * time_step
    else:
        raise ValueError("Neither 'Time' nor 'Step' column found.")
    
    # Average over the equilibrium window if provided.
    if eq_window is not None:
        total_time = time_data[-1]
        eq_mask = time_data >= (total_time - eq_window)
        avg_enthalpy = np.mean(enthalpy[eq_mask])
    else:
        avg_enthalpy = np.mean(enthalpy)
    
    # Convert from eV/box to meV/mol.
    if num_molecules is None:
        raise ValueError("num_molecules must be provided for conversion to meV/mol.")
    # Average per molecule in eV.
    avg_per_molecule = avg_enthalpy / num_molecules
    # Convert to eV per mole.
    NA = 6.02214076e23  # Avogadro's number (molecules/mol)
    avg_per_mole_eV = avg_per_molecule 
    # Convert to meV.
    avg_enthalpy_meVmol = avg_per_mole_eV * 1000
    
    return avg_enthalpy_meVmol

## Analysis Functions 
## 4 - Equilibrium Density

In [5]:
def read_lammps_log(filename, time_step=None):
    """
    Reads a LAMMPS log file and extracts the numerical data.

    Parameters:
    filename (str): Path to the log file.
    time_step (float): Time per step (in ps), optional.

    Returns:
    header (list of str): List of column names.
    data (np.ndarray): 2D numpy array of the numerical data.
    """
    data_lines = []
    header = None
    with open(filename, 'r') as file:
        lines = file.readlines()
        i = 0
        while i < len(lines):
            line = lines[i]
            if line.strip().startswith('Step'):
                header_line = line
                header = line.strip().split()
                data_start_index = i + 1
                i += 1
                # Read data lines until we hit an empty line or a non-data line
                while i < len(lines):
                    line = lines[i].strip()
                    if not line:
                        break  # Empty line signifies end of data block
                    tokens = line.split()
                    # Check if the number of tokens matches the header length
                    if len(tokens) != len(header):
                        break  # Likely end of data block
                    try:
                        data_line = [float(token) for token in tokens]
                        data_lines.append(data_line)
                    except ValueError:
                        break  # Non-numeric line, end of data block
                    i += 1
            else:
                i += 1
    if not data_lines:
        raise ValueError('No data found in the log file.')
    data = np.array(data_lines)
    # Now, check if 'Time' is in header, if not, compute it
    if 'Time' not in header:
        if time_step is None:
            time_step = 0.0005  # Adjust as necessary based on your simulation parameters
        # Compute 'Time' from 'Step'
        if 'Step' not in header:
            raise ValueError("'Step' column not found in data.")
        step_idx = header.index('Step')
        time_array = data[:, step_idx] * time_step
        # Add 'Time' column to header and data
        header.append('Time')
        data = np.hstack((data, time_array.reshape(-1, 1)))
    return header, data

def read_ase_log(filename):
    """
    Reads an ASE log file and extracts the numerical data, adjusting 'Step' values to be continuous across restarts.

    Parameters:
    filename (str): Path to the ASE log file.

    Returns:
    header (list of str): List of column names.
    data (np.ndarray): 2D numpy array of the numerical data.
    """
    data_lines = []
    header = ['Step', 'Time', 'Temp', 'Press', 'Density', 'TotEng']
    time_step = 0.0005  # Adjust based on your actual time step (ps per step)
    step_offset = 0
    prev_step = None
    with open(filename, 'r') as file:
        lines = file.readlines()
        for line_num, line in enumerate(lines):
            line = line.strip()
            if line.startswith('Step:'):
                # Replace missing spaces or commas
                line = re.sub(r'([a-zA-Z]):', r'\1: ', line)
                line = re.sub(r'([a-zA-Z])([0-9])', r'\1 \2', line)
                # Split the line into key-value pairs
                parts = line.split(',')
                data_dict = {}
                for part in parts:
                    if ':' in part:
                        key, value = part.split(':', 1)
                        key = key.strip()
                        value = value.strip()
                        # Remove units and extract numerical value
                        value = value.split()[0]
                        try:
                            data_dict[key] = float(value)
                        except ValueError:
                            continue
                # Parse current step
                current_step = data_dict.get('Step', 0)
                # Check for restart (Step reset)
                if prev_step is not None and current_step < prev_step:
                    # Restart detected
                    step_offset += prev_step + 1  # Add offset
                # Adjust step
                adjusted_step = current_step + step_offset
                prev_step = current_step
                # Calculate time
                time = adjusted_step * time_step
                # Handle density unit conversion (if needed)
                density = data_dict.get('Density', np.nan)
                if not np.isnan(density):
                    # Assuming ASE density is in correct units already
                    density = density 
                data_line = [
                    adjusted_step,
                    time,
                    data_dict.get('Temperature', np.nan),
                    data_dict.get('Pressure', np.nan),
                    density,
                    data_dict.get('Total Energy', np.nan)
                ]
                data_lines.append(data_line)
    if not data_lines:
        raise ValueError('No data found in the ASE log file.')
    data = np.array(data_lines)
    return header, data

def running_mean_std(data, window_size):
    """
    Calculate the running mean and standard deviation using a sliding window.

    Parameters:
    data (np.ndarray): 1D array of data points
    window_size (int): Size of the sliding window

    Returns:
    mean (np.ndarray): Running mean
    std (np.ndarray): Running standard deviation
    """
    cumsum = np.cumsum(np.insert(data, 0, 0))
    cumsum_sq = np.cumsum(np.insert(data**2, 0, 0))

    mean = (cumsum[window_size:] - cumsum[:-window_size]) / window_size
    mean_sq = (cumsum_sq[window_size:] - cumsum_sq[:-window_size]) / window_size
    variance = mean_sq - mean**2
    variance[variance < 0] = 0  # Correct for small negative variances due to floating-point errors
    std = np.sqrt(variance)
    return mean, std
def analyze_and_plot_logs(log_files, window_size, column_to_analyze, labels, colors, output_filename, plot_title, 
                              eq_window, total_sim_time):
    """
    Reads multiple log files (LAMMPS and ASE), calculates running mean and std for specified column,
    and plots the results. Also computes an equilibrium average over a specified time window.

    Parameters:
    log_files (list of tuples): List of tuples containing the log type ('lammps' or 'ase') and file path.
    window_size (int): Window size for running mean and std.
    column_to_analyze (str): Name of the column to analyze (e.g., 'Press', 'Density').
    labels (list of str): Labels for each dataset (used in the legend).
    colors (list of str): Colors for each dataset.
    output_filename (str): Filename to save the plot.
    plot_title (str): Title of the plot.
    eq_window (float): Time window (in ps) at end of simulation for equilibrium calculation.
    total_sim_time (float): Total simulation time (in ps).
    """
    data_list = []
    for i, (log_type, filename) in enumerate(log_files):
        if log_type == 'lammps':
            header, data = read_lammps_log(filename)
        elif log_type == 'ase':
            header, data = read_ase_log(filename)
        else:
            raise ValueError(f"Unknown log type '{log_type}' for file '{filename}'")
        data_list.append((header, data))

    adjusted_time_arrays = []
    mean_arrays = []
    std_arrays = []
    eq_means = []
    eq_stds = []

    for i, (header, data) in enumerate(data_list):
        column_indices = {name: idx for idx, name in enumerate(header)}
        if column_to_analyze not in column_indices:
            raise ValueError(f"Column '{column_to_analyze}' not found in data for dataset '{labels[i]}'")

        # Determine time array
        if 'Time' in column_indices:
            time_data = data[:, column_indices['Time']]
        else:
            if 'Step' not in column_indices:
                raise ValueError(f"'Time' or 'Step' column not found in data for dataset '{labels[i]}'")
            time_step = 0.0005
            time_data = data[:, column_indices['Step']] * time_step

        col_data = data[:, column_indices[column_to_analyze]]
        if len(col_data) < window_size:
            raise ValueError(f"Not enough data points in '{labels[i]}' for the given window size.")

        # Compute running mean and std
        mean_col, std_col = running_mean_std(col_data, window_size)
        mean_arrays.append(mean_col)
        std_arrays.append(std_col)
        adjusted_time_arrays.append(time_data[window_size - 1:])

        # Equilibrium calculation
        eq_start_time = total_sim_time - eq_window
        eq_mask = time_data >= eq_start_time
        eq_data = col_data[eq_mask]
        if eq_data.size > 0:
            eq_mean = np.mean(eq_data)
            eq_std = np.std(eq_data)
        else:
            eq_mean = np.nan
            eq_std = np.nan
        eq_means.append(eq_mean)
        eq_stds.append(eq_std)

        # Print equilibrium values
        print(f"Equilibrium stats for '{labels[i]}': mean = {eq_mean:.4f}, std = {eq_std:.4f}")

    # Plotting
    plt.figure()
    for i in range(len(mean_arrays)):
        time_arr = adjusted_time_arrays[i]
        plt.plot(time_arr, mean_arrays[i], label=labels[i], color=colors[i])
        plt.fill_between(time_arr,
                         mean_arrays[i] - std_arrays[i],
                         mean_arrays[i] + std_arrays[i],
                         color=colors[i], alpha=0.2)
        # Equilibrium line
        eq_start_time = total_sim_time - eq_window
        plt.hlines(eq_means[i], eq_start_time, total_sim_time,
                   colors=colors[i], linestyles='--',
                   label=f"{labels[i]} eq mean = {eq_means[i]:.4f} ± {eq_stds[i]:.4f}")

    plt.xlabel('Time (ps)')
    plt.ylabel(column_to_analyze)
    plt.title(plot_title, size=14)
    plt.legend(loc='best')
    plt.xlim(0, max(arr[-1] for arr in adjusted_time_arrays))
    plt.savefig(output_filename, dpi=300)
    plt.close()

## Analysis Functions 
## 5 - Spectrum

In [6]:
def read_autocorrelation_from_csv(csv_filename):
    """
    Loads the CSV file containing velocity auto-correlation function 
    as obtained from the do_vacf.py script.
    """
    freq_vals = []
    T_vals = []
    VACF_vals = []

    # 1) Read data from CSV
    with open(csv_filename, 'r', newline='') as f:
        reader = csv.reader(f)
        header = next(reader)  # skip the header row
        # Expect something like ["Frequency(cm^-1)", "GaussSpectrum", "HannSpectrum"]

        for row in reader:
            # Each 'row' is a list of strings from that line
            # Convert them to float
            T_vals.append(float(row[0]))
            VACF_vals.append(float(row[1]))

    # 2) Convert to NumPy arrays (not strictly required, but often convenient)
    T_vals = np.array(freq_vals)
    VACF_vals = np.array(VACF_vals)
    return T_vals, VACF_vals

def hann_windowed_fft_real_power(vacf, dt_fs, truncation_length=None):
    """
    Applies Hann window to a truncated VACF segment and computes FFT.
    
    Parameters:
    - vacf: 1D numpy array of the velocity autocorrelation function
    - dt_fs: timestep in femtoseconds
    - truncation_length: number of points to keep from the VACF before applying the Hann window.
                         If None, use the full VACF length.
    
    Returns:
    - freq_cm: frequencies in cm^-1
    - ps_real_only: square of real part of FFT
    - abs_part: absolute value of FFT (useful for standard power spectrum)
    """
    
    if truncation_length is None or truncation_length > len(vacf):
        truncation_length = len(vacf)

    vacf_cut = vacf[:truncation_length]
    
    # Apply Hann window
    window = hann(truncation_length)
    vacf_windowed = vacf_cut * window

    # Compute FFT
    spec = np.fft.rfft(vacf_windowed)

    real_part = np.abs(spec)**2
    abs_part = np.abs(spec)

    # Frequency axis in fs^-1
    freq_fs_inv = np.fft.rfftfreq(truncation_length, d=dt_fs)

    # Convert to cm^-1
    factor_fs_inv_to_cm = 3.335640951e4
    freq_cm = freq_fs_inv * factor_fs_inv_to_cm

    return freq_cm, real_part, abs_part

def apply_qcf(freq, spectrum, temperature):
    """
    Apply a quantum correction factor (QCF) to the power spectrum.
   
    Parameters:
        freq (np.ndarray): Frequency array from FFT.
        spectrum (np.ndarray): Power spectrum to be corrected.
        temperature (float): Temperature in Kelvin (for quantum correction).
       
    Returns:
        np.ndarray: Corrected power spectrum.
    """
    hbar = 1.0545718e-34  # Planck's constant (J·s)
    kB = 1.380649e-23     # Boltzmann constant (J/K)
    beta = hbar * freq / (kB * temperature)
    correction = beta / (1 - np.exp(-beta))
    corrected_spectrum = spectrum * correction
    return corrected_spectrum

def gaussian_broaden(freq_cm, spectrum, fwhm_cm):
    """
    Convolve 'spectrum' with a Gaussian of a given FWHM in cm^-1.

    freq_cm : 1D array of frequencies in cm^-1 (assumed evenly spaced)
    spectrum: 1D array (same length as freq_cm)
    fwhm_cm : Desired full-width at half-maximum (FWHM) in cm^-1 for broadening

    Returns:
      spectrum_broadened : 1D array (same length as freq_cm) 
                           after applying Gaussian broadening
    """
    # 1) Determine the spacing between frequency points (should be constant if using rfft)
    df = freq_cm[1] - freq_cm[0]  # cm^-1 step

    # 2) Convert the user-specified FWHM in cm^-1 to a Gaussian sigma (std dev) in cm^-1
    #    For a Gaussian: FWHM = 2.35482 * sigma
    sigma_cm = fwhm_cm / 2.35482

    # 3) Convert sigma from cm^-1 to array indices
    sigma_pts = sigma_cm / df

    # 4) Use scipy's 1D Gaussian filter (which operates in array index space)
    #    mode='reflect' or 'nearest' are typical choices to handle edges
    spectrum_broadened = gaussian_filter1d(spectrum, sigma=sigma_pts, mode='reflect')

    return spectrum_broadened

## Analysis Functions 
## 6 Heat Capacity

In [7]:
def read_lammps_log(filename, time_step=None):
    """Read a LAMMPS log file and return (header, data array)."""
    data_lines, header = [], None
    with open(filename, 'r') as file:
        lines = file.readlines()
        i = 0
        while i < len(lines):
            if lines[i].strip().startswith('Step'):
                header = lines[i].strip().split()
                i += 1
                while i < len(lines) and lines[i].strip():
                    tokens = lines[i].split()
                    if len(tokens) != len(header):
                        break
                    try:
                        data_lines.append([float(tok) for tok in tokens])
                    except ValueError:
                        break
                    i += 1
            else:
                i += 1
    if not data_lines:
        raise ValueError(f"No data found in {filename}")
    data = np.array(data_lines)
    # add Time column if missing
    if "Time" not in header:
        if "Step" not in header:
            raise ValueError("'Step' column not found in log file.")
        if time_step is None:
            time_step = 0.0005  # ps per step, adjust if needed
        step_idx = header.index("Step")
        time_array = data[:, step_idx] * time_step
        header.append("Time")
        data = np.hstack((data, time_array.reshape(-1, 1)))
    return header, data

# ---------------- Main analysis ----------------
def compute_cp(
    log_files,
    n_molecules=510,
    molecular_weight=18.01528,
    quantum_corr=6.0,
    fit="poly",
    degree=3,
    plot=True,
):
    """
    Compute Cp(T) from a list of LAMMPS log files.

    Parameters
    ----------
    log_files : list of str
        Paths to log files.
    n_molecules : int
        Number of molecules in the box.
    molecular_weight : float
        Molecular weight in g/mol (default water).
    quantum_corr : float
        Quantum correction (cal/mol/K) to subtract at 298 K.
    fit : str
        'poly' for polynomial fit, 'spline' for UnivariateSpline.
    degree : int
        Polynomial degree (ignored if fit='spline').
    plot : bool
        If True, make a Cp vs T plot.

    Returns
    -------
    T_fine : ndarray
        Temperature grid.
    cp_fine : ndarray
        Cp(T) in cal/mol/K.
    """

    T_list, H_list = [], []

    for path in log_files:
        hdr, data = read_lammps_log(path)
        cut = len(data) // 2  # discard equilibration
        T0   = data[cut:, hdr.index("Temp")].mean()
        P0   = data[cut:, hdr.index("Press")].mean()
        rho0 = data[cut:, hdr.index("Density")].mean()
        E0_eV= data[cut:, hdr.index("E_pair")].mean()

        # Energy per molecule (eV → kcal/mol)
        eV_per_mol = E0_eV / n_molecules
        J_per_mol  = eV_per_mol * 1.602176634e-19 * 6.02214076e23
        E0_kcal    = J_per_mol / 4184.0

        # PV term (kcal/mol)
        V_mol_cm3 = molecular_weight / rho0
        PV_kcal   = P0 * V_mol_cm3 * 0.101325 / 4184.0

        H0_kcal = E0_kcal + PV_kcal
        T_list.append(T0)
        H_list.append(H0_kcal * 1000.0)  # → cal/mol

    # Sort by T
    T_arr, H_arr = np.array(T_list), np.array(H_list)
    order = np.argsort(T_arr)
    T_arr, H_arr = T_arr[order], H_arr[order]

    # Fit H(T)
    T_fine = np.linspace(T_arr.min(), T_arr.max(), 400)
    if fit == "poly":
        coefs = np.polyfit(T_arr, H_arr, degree)
        H_fine = np.polyval(coefs, T_fine)
        cp_fine = np.polyval(np.polyder(coefs), T_fine)
    elif fit == "spline":
        spl = UnivariateSpline(T_arr, H_arr, k=5, s=1)
        H_fine = spl(T_fine)
        cp_fine = spl.derivative()(T_fine)
    else:
        raise ValueError("fit must be 'poly' or 'spline'")

    # Quantum correction near 298 K
    i298 = np.abs(T_fine - 298.15).argmin()
    cp_fine[i298] -= quantum_corr

    # Plot
    if plot:
        plt.figure(figsize=(3.3, 2.8), dpi=300)
        plt.plot(T_fine, cp_fine, label="Classical")
        plt.xlabel("T (K)")
        plt.ylabel(r"$c_p$ (cal/mol K)")
        plt.legend(frameon=False, fontsize=8)
        plt.tight_layout()
        plt.show()

    return T_fine, cp_fine


## Analysis Functions 
## 7 Thermal Expansion Coefficient

In [ ]:
#insert T and rho lists from npt equilibrations
T_list = []
rho_list = []
# convert to sorted numpy arrays (they’re already in ascending T, but this is a safeguard)
T_arr   = np.array(T_list)
rho_arr = np.array(rho_list)
order   = np.argsort(T_arr)
T_arr   = T_arr[order]
rho_arr = rho_arr[order]

# ————— spline fit & derivative —————
# choose a small smoothing factor s if your data are noisy
spl_rho     = UnivariateSpline(T_arr, rho_arr, k=5, s=1)
drho_dT     = spl_rho.derivative()(T_arr)   # (g/cm^3)/K
alphaP      = - drho_dT / rho_arr          # in K^(-1)
